In [1]:
import numpy as np
import logging
import pyLDAvis.gensim
import json
import warnings

In [2]:
# add log
warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarit

In [3]:
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from numpy import array

In [4]:
# set up logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


In [5]:
# set up corpus
texts = [['oha', 'hvva', 'hv', 'ah', 'ah', 'ch', 'a', 'h', 'ajc', 'va', 'ohx', 'ajcva', 'oa', 'h', 'va', 'o', 'qjch', 'vcoa', 'h', 'qajc', 'v', 'h', 'ha', 'ha', 'va', 'h', 'a', 'h', 'a', 'h', 'qoo'], ['o', 'oo', 'oooh', 'aoah', 'aha', 'oh', 'aa', 'h', 'a', 'h', 'vv', 'a', 'ahh', 'oao', 'a', 'o', 'ho', 'o', 'o', 'o', 'ojcojc', 'jcx', 'h', 'xa', 'o', 'a', 'hah', 'jcj', 'cxa', 'a', 'h', 'ah', 'ah', 'aho', 'jccjcca', 'qj', 'a', 'oa', 'ho', 'haa', 'oooo', 'oa', 'ojxcoo', 'o', 'ha', 'o', 'hh', 'a', 'a', 'h', 'a', 'h', 'ah', 'a', 'h', 'aq', 'qo', 'o'], ['h', 'haa', 'oahah', 'oahoo', 'iahah', 'hbibh', 'hp', 'bvbhoo', 'oah', 'aohaoo', 'a', 'hah', 'ao', 'obhb', 'hoa', 'ao', 'a', 'ojcch', 'o', 'ah', 'bh'], ['hah', 'ao', 'o', 'ahah', 'hhhah', 'h', 'hhahah', 'h', 'b', 'aaaa', 'a', 'ahha', 'a', 'a', 'a', 'a', 'aa', 'aa', 'aba', 'a', 'a', 'ah', 'a', 'a', 'a', 'ah', 'bhaha', 'hh', 'hihh', 'a', 'aa', 'aaha', 'ab'], ['oaoo', 'ohao', 'aoao', 'aoao', 'aha', 'oh', 'a', 'h', 'a', 'h', 'a', 'oa', 'oa', 'hoao', 'hao', 'ha', 'h', 'a', 'vva', 'oa', 'ahaah', 'aha', 'ho', 'aoaoo', 'o', 'oo', 'oah', 'hoha', 'hv', 'aoa', 'oaooo', 'aoa', 'oib', 'jcoqq', 'qjaha', 'ajaajwa', 'q', 'va', 'b', 'oh', 'o', 'oho', 'piaq', 'jj', 'v', 'jxaq', 'ia', 'iajaia', 'qjcxqa', 'oojxc', 'q', 'bioapa', 'pa', 'ho', 'va', 'hoib', 'ha', 'h', 'c', 'h', 'h', 'b', 'h', 'ch', 'ah', 'j', 'bo', 'ivjcp', 'jah', 'bv', 'ho']]


dictionary = Dictionary(texts)

# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(101 unique tokens: ['a', 'ah', 'ajc', 'ajcva', 'ch']...) from 5 documents (total 214 corpus positions)


In [6]:
# set up two topic models
goodLdaModel = LdaModel(corpus=corpus, id2word=dictionary, iterations=50, num_topics=4)
badLdaModel = LdaModel(corpus=corpus, id2word=dictionary, iterations=1, num_topics=4)

INFO:gensim.models.ldamodel:using symmetric alpha at 0.25
INFO:gensim.models.ldamodel:using symmetric eta at 0.25
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running online (single-pass) LDA training, 4 topics, 1 passes over the supplied corpus of 5 documents, updating model once every 5 documents, evaluating perplexity every 5 documents, iterating 50x with a convergence threshold of 0.001000
DEBUG:gensim.models.ldamodel:bound: at document #0
INFO:gensim.models.ldamodel:-6.218 per-word bound, 74.5 perplexity estimate based on a held-out corpus of 5 documents with 214 words
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #5/5
DEBUG:gensim.models.ldamodel:performing inference on a chunk of 5 documents
DEBUG:gensim.models.ldamodel:1/5 documents converged within 50 iterations
DEBUG:gensim.models.ldamodel:updating topics
INFO:gensim.models.ldamodel:topic #0 (0.250): 0.169*"a" + 0.058*"h" + 0.057*"o" + 0.037*"aa" + 0.037*"ah" + 0.02

In [7]:
# using U_Mass Coherence
goodcm = CoherenceModel(model=goodLdaModel, corpus=corpus, dictionary=dictionary, coherence='u_mass')
badcm = CoherenceModel(model=badLdaModel, corpus=corpus, dictionary=dictionary, coherence='u_mass')

DEBUG:gensim.models.coherencemodel:Setting topics to those of the model: LdaModel(num_terms=101, num_topics=4, decay=0.5, chunksize=2000)
DEBUG:gensim.models.coherencemodel:Setting topics to those of the model: LdaModel(num_terms=101, num_topics=4, decay=0.5, chunksize=2000)


In [8]:
# view
print(goodcm)

Coherence_Measure(seg=<function s_one_pre at 0x000001CD2F626400>, prob=<function p_boolean_document at 0x000001CD2F626598>, conf=<function log_conditional_probability at 0x000001CD2F6EA2F0>, aggr=<function arithmetic_mean at 0x000001CD2F6EAA60>)


In [9]:
# Visualize topic models
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(goodLdaModel, corpus, dictionary)


DEBUG:gensim.models.ldamodel:performing inference on a chunk of 5 documents
DEBUG:gensim.models.ldamodel:3/5 documents converged within 50 iterations


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      42.574947        1       1  0.030143 -0.017612
0      38.335915        1       2 -0.046241 -0.003468
2      12.841225        1       3  0.022483  0.017497
3       6.247913        1       4 -0.006384  0.003583, topic_info=     Category       Freq     Term      Total  loglift  logprob
term                                                          
9     Default  10.000000        o  10.000000  30.0000  30.0000
0     Default  23.000000        a  23.000000  29.0000  29.0000
5     Default  17.000000        h  17.000000  28.0000  28.0000
19    Default   3.000000       aa   3.000000  27.0000  27.0000
29    Default   3.000000       ho   3.000000  26.0000  26.0000
10    Default   4.000000       oa   4.000000  25.0000  25.0000
34    Default   2.000000       oh   2.000000  24.0000  24.0000
20    Default   2.000000      aha   2.000000  23.0000  23.0000
17    Default   4.000000       va   4.000000  22.0000  22.0000
6     Default   4.000000       ha   4.000000  21.0000  21.0000
28    Default   2.000000       hh   2.000000  20.0000  20.0000
37    Default   1.000000       oo   1.000000  19.0000  19.0000
30    Default   1.000000  jccjcca   1.000000  18.0000  18.0000
44    Default   2.000000       ao   2.000000  17.0000  17.0000
40    Default   1.000000       qj   1.000000  16.0000  16.0000
25    Default   1.000000      cxa   1.000000  15.0000  15.0000
32    Default   1.000000      jcx   1.000000  14.0000  14.0000
31    Default   1.000000      jcj   1.000000  13.0000  13.0000
26    Default   1.000000      haa   1.000000  12.0000  12.0000
36    Default   1.000000   ojxcoo   1.000000  11.0000  11.0000
41    Default   1.000000       qo   1.000000  10.0000  10.0000
38    Default   1.000000     oooh   1.000000   9.0000   9.0000
22    Default   1.000000      aho   1.000000   8.0000   8.0000
35    Default   1.000000   ojcojc   1.000000   7.0000   7.0000
71    Default   1.000000     aoao   1.000000   6.0000   6.0000
43    Default   1.000000       xa   1.000000   5.0000   5.0000
33    Default   1.000000      oao   1.000000   4.0000   4.0000
23    Default   1.000000     aoah   1.000000   3.0000   3.0000
39    Default   1.000000     oooo   1.000000   2.0000   2.0000
42    Default   1.000000       vv   1.000000   1.0000   1.0000
...       ...        ...      ...        ...      ...      ...
22     Topic4   0.162214      aho   1.254665   0.7272  -4.4119
35     Topic4   0.160806   ojcojc   1.245740   0.7256  -4.4206
43     Topic4   0.159315       xa   1.255303   0.7087  -4.4299
33     Topic4   0.155346      oao   1.258998   0.6805  -4.4552
23     Topic4   0.155117     aoah   1.262015   0.6766  -4.4566
39     Topic4   0.152278     oooo   1.263960   0.6566  -4.4751
42     Topic4   0.148645       vv   1.262289   0.6338  -4.4992
24     Topic4   0.148559       aq   1.271214   0.6262  -4.4998
21     Topic4   0.143863      ahh   1.288028   0.5809  -4.5319
37     Topic4   0.183557       oo   1.917942   0.4264  -4.2883
9      Topic4   0.934686        o  10.163074   0.3866  -2.6606
26     Topic4   0.166669      haa   1.942577   0.3172  -4.3848
28     Topic4   0.171165       hh   2.059147   0.2855  -4.3582
20     Topic4   0.203318      aha   2.576231   0.2336  -4.1860
34     Topic4   0.200539       oh   2.553609   0.2287  -4.1998
93     Topic4   0.093126      oib   1.280842   0.1516  -4.9669
92     Topic4   0.092046      oho   1.273638   0.1456  -4.9785
82     Topic4   0.090066   iajaia   1.272903   0.1244  -5.0003
75     Topic4   0.088035       bv   1.249678   0.1200  -5.0231
83     Topic4   0.089426    ivjcp   1.273800   0.1166  -5.0074
80     Topic4   0.087555     hoib   1.250809   0.1136  -5.0285
87     Topic4   0.087293       jj   1.247577   0.1132  -5.0315
69     Topic4   0.089707  ajaajwa   1.282706   0.1127  -5.0043
29     Topic4   0.265244       ho   3.911742   0.0818  -3.9202
10     Topic4   0.281176       oa   4.7156

In [10]:
# Visualize topic models
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(badLdaModel, corpus, dictionary)


DEBUG:gensim.models.ldamodel:performing inference on a chunk of 5 documents
DEBUG:gensim.models.ldamodel:0/5 documents converged within 1 iterations


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      26.924875        1       1 -0.000062  0.001495
3      26.563904        1       2 -0.002375  0.000162
0      23.923773        1       3 -0.000176 -0.001724
2      22.587450        1       4  0.002613  0.000067, topic_info=     Category       Freq     Term      Total  loglift  logprob
term                                                          
0     Default  21.000000        a  21.000000  30.0000  30.0000
44    Default   2.000000       ao   2.000000  29.0000  29.0000
29    Default   4.000000       ho   4.000000  28.0000  28.0000
52    Default   2.000000      oah   2.000000  27.0000  27.0000
97    Default   2.000000        q   2.000000  26.0000  26.0000
20    Default   2.000000      aha   2.000000  25.0000  25.0000
6     Default   4.000000       ha   4.000000  24.0000  24.0000
5     Default  17.000000        h  17.000000  23.0000  23.0000
71    Default   2.000000     aoao   2.000000  22.0000  22.0000
93    Default   1.000000      oib   1.000000  21.0000  21.0000
56    Default   1.000000    ojcch   1.000000  20.0000  20.0000
26    Default   2.000000      haa   2.000000  19.0000  19.0000
55    Default   1.000000     obhb   1.000000  18.0000  18.0000
70    Default   2.000000      aoa   2.000000  17.0000  17.0000
47    Default   1.000000   bvbhoo   1.000000  16.0000  16.0000
60    Default   1.000000      aba   1.000000  15.0000  15.0000
49    Default   1.000000      hoa   1.000000  14.0000  14.0000
88    Default   1.000000     jxaq   1.000000  13.0000  13.0000
76    Default   1.000000        c   1.000000  12.0000  12.0000
85    Default   1.000000      jah   1.000000  11.0000  11.0000
69    Default   1.000000  ajaajwa   1.000000  10.0000  10.0000
96    Default   1.000000     piaq   1.000000   9.0000   9.0000
51    Default   1.000000    iahah   1.000000   8.0000   8.0000
86    Default   1.000000    jcoqq   1.000000   7.0000   7.0000
34    Default   2.000000       oh   2.000000   6.0000   6.0000
80    Default   1.000000     hoib   1.000000   5.0000   5.0000
54    Default   1.000000    oahoo   1.000000   4.0000   4.0000
9     Default  10.000000        o  10.000000   3.0000   3.0000
75    Default   1.000000       bv   1.000000   2.0000   2.0000
95    Default   1.000000       pa   1.000000   1.0000   1.0000
...       ...        ...      ...        ...      ...      ...
95     Topic4   0.346830       pa   1.349527   0.1291  -4.9371
51     Topic4   0.346086    iahah   1.347144   0.1287  -4.9393
94     Topic4   0.346945    oojxc   1.351836   0.1277  -4.9368
90     Topic4   0.346786    oaooo   1.351992   0.1271  -4.9372
54     Topic4   0.344661    oahoo   1.347980   0.1240  -4.9434
87     Topic4   0.344541       jj   1.350910   0.1215  -4.9437
84     Topic4   0.341362        j   1.350614   0.1124  -4.9530
46     Topic4   0.339279       bh   1.349988   0.1067  -4.9591
81     Topic4   0.339159       ia   1.351817   0.1050  -4.9595
80     Topic4   0.339520     hoib   1.355596   0.1033  -4.9584
73     Topic4   0.336297   bioapa   1.353353   0.0954  -4.9680
0      Topic4   4.809558        a  21.011189   0.0133  -2.3076
9      Topic4   2.277815        o  10.203861  -0.0118  -3.0550
16     Topic4   0.492535        v   2.029263   0.0719  -4.5864
29     Topic4   0.928029       ho   4.073510   0.0086  -3.9529
17     Topic4   0.926687       va   4.073475   0.0071  -3.9543
5      Topic4   3.337715        h  17.101274  -0.1461  -2.6729
1      Topic4   1.452674       ah   6.823018  -0.0591  -3.5048
34     Topic4   0.614753       oh   2.724592  -0.0011  -4.3647
10     Topic4   0.934567       oa   4.795960  -0.1477  -3.9459
63     Topic4   0.600772        b   2.728758  -0.0256  -4.3877
27     Topic4   0.596694      hah   2.718722  -0.0287  -4.3946
37     Topic4   0.469742       oo   2.034748   0.0218  -4.6338
19     Topic4   0.659858       aa   3.425210  -0.1591  -4.2939
71     Topic4   0.462561     aoao   2.0410